# **Titanic**

The objective of this notebook is to predict whether a passenger onboard the titanic will survive given features such as their gender, age, class etc.

## 1. Preparations

We will begin by installing all of the necessary libraries and loading in the data. Since this project is for a Kaggle competition, I will be using the Kaggle API to download the data and submit my results.

### 1.1. Dependencies

In [ ]:
!pip install kaggle 
!pip install scikit-learn

In [ ]:
import kaggle 
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

api.competition_download_file('titanic', 'train.csv')
api.competition_download_file('titanic', 'test.csv')

In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

### 1.2. Importing Data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train.shape, test.shape

((891, 12), (418, 11))

## 2. Preprocessing Training Data

We can now begin cleaning the data so all of the relevant features are included and formatted in a suitable way to run a machine learning model.

### 2.1. Removing Extraneous Features

In [6]:
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)
train.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


### 2.2. Remove Missing Data

In [7]:
print(train.shape)
train.isnull().sum()

(891, 8)


Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

As can be seen, there are only 2 passengers with an unknown departure point but the ages of 177 passengers are missing. Since this is ~ 20% of the full dataset, missing value imputation needs to be conducted. But first, I will remove the two passengers with missing embarkment locations.

In [8]:
train.dropna(subset = ['Embarked'], inplace = True)
train.shape

(889, 8)

In [9]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### 2.3. One-Hot Encoding

Three of the variables provided are categorical ('Pclass', 'Sex', 'Embarked') and therefore need to be converted into dummy variables. We therefore will have 12 features in the final machine learnign model.

In [10]:
one_hot = pd.get_dummies(train['Pclass'])
train = train.drop(['Pclass'],axis = 1)
train = train.join(one_hot)

one_hot = pd.get_dummies(train['Sex'])
train = train.drop(['Sex'],axis = 1)
train = train.join(one_hot)

one_hot = pd.get_dummies(train['Embarked'])
train = train.drop(['Embarked'],axis = 1)
train = train.join(one_hot)

In [11]:
train.head()

,Survived,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
0,0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
2,1,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
4,0,35.0,0,0,8.0500,0,0,1,0,1,0,0,1


### 2.4. Data Imputation

Multiple imputation by chained equations (MICE) was used to fill in values for the unknown ages of 177 passengers. This method involves multiple rounds of imputation. The first round inputs a placeholder value (e.g. mean value of known ages) and then uses the other variables as factors in a Bayesian regression model. This model can be used to provide updated estimates of the age before repeating the process again. Since there is no missing data for any of the other variables, this method should work quite well. 

In [12]:
# using fancy impute for MICE
mice_imputer = IterativeImputer()
train_arr = mice_imputer.fit_transform(train)

# convert numpy array to pd
full_train = pd.DataFrame(train_arr, columns = ['Survived', 'Age', 'SibSp', 'Parch', 'Fare', '1', '2', '3', 'female', 'male', 'C', 'Q', 'S'])
full_train.head(10)

,Survived,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
0,0.0,22.000000,1.0,0.0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,38.000000,1.0,0.0,71.2833,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,26.000000,0.0,0.0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.0,35.000000,1.0,0.0,53.1000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,35.000000,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
5,0.0,31.324242,0.0,0.0,8.4583,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,0.0,54.000000,0.0,0.0,51.8625,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7,0.0,2.000000,3.0,1.0,21.0750,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,1.0,27.000000,0.0,2.0,11.1333,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
9,1.0,14.000000,1.0,0.0,30.0708,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


## 3. Preprocessing Test Data

We now need to prepare the test dataset for the models. Many of the previous steps will be repeated 

### 3.1. Removing Extraneous Features

In [13]:
test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)
test.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
5,3,male,14.0,0,0,9.2250,S
6,3,female,30.0,0,0,7.6292,Q
7,2,male,26.0,1,1,29.0000,S
8,3,female,18.0,0,0,7.2292,C
9,3,male,21.0,2,0,24.1500,S


### 3.2. One-Hot Encoding

In [14]:
one_hot = pd.get_dummies(test['Pclass'])
test = test.drop(['Pclass'],axis = 1)
test = test.join(one_hot)

one_hot = pd.get_dummies(test['Sex'])
test = test.drop(['Sex'],axis = 1)
test = test.join(one_hot)

one_hot = pd.get_dummies(test['Embarked'])
test = test.drop(['Embarked'],axis = 1)
test = test.join(one_hot)

### 3.3. Data Imputation

In [15]:
# using fancy impute for MICE
mice_imputer = IterativeImputer()
test_arr = mice_imputer.fit_transform(test)

# convert numpy array to pd
full_test = pd.DataFrame(test_arr, columns = ['Age', 'SibSp', 'Parch', 'Fare', '1', '2', '3', 'female', 'male', 'C', 'Q', 'S'])
full_test.head(10)

,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
0,34.5,0.0,0.0,7.8292,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,47.0,1.0,0.0,7.0000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,62.0,0.0,0.0,9.6875,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,27.0,0.0,0.0,8.6625,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,22.0,1.0,1.0,12.2875,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
5,14.0,0.0,0.0,9.2250,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6,30.0,0.0,0.0,7.6292,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
7,26.0,1.0,1.0,29.0000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
8,18.0,0.0,0.0,7.2292,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
9,21.0,2.0,0.0,24.1500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


## 4. Modelling

Since we are predicting whether a passenger survived or not (1 or 0), the response variable is binary. We can therefore apply a range of classification or regression techniques and submit the results for the model which has the highest accuracy in the cross-validation process. But before we can do that, we need to split our training data into a further train and test dataset. I will split the data into 75% training and 25% testing. 

In [16]:
training, testing = train_test_split(full_train, test_size=0.25)
x_training = training.drop('Survived', axis = 1)
y_training = training.Survived

x_testing = testing.drop('Survived', axis = 1)
y_testing = testing.Survived

...Then finally we will want to store each model's accuracy in a dictionary

In [46]:
comparison_dict ={'model':[],
                  'params': [],
                  'k-fold accuracy': [],
                  'validate accuracy': []}

### 4.1. XGBoost

The first model I attempt will be an extreme gradient boosting model. This is because I think it is the most likely to be the best performing due to it being a very fast, ensemble model that uses regularisation  

#### 4.1.1. Model Fitting

In [67]:
# setting parameters for hyperparameter tuning
params = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.05, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse"],
    "subsample":[0.5, 0.618, 0.85, 0.95, 1.0],
    "n_estimators":[10]
    }

# fitting the model
model = GridSearchCV(GradientBoostingClassifier(), params, cv = 5, n_jobs = -1)
model.fit(x_training, y_training)
print("The cross-validation model score was {}".format(model.score(x_training, y_training)))
print(model.best_score_)
print(model.best_params_)

# validation
y_pred = model.predict(x_testing)
validate_accuracy = accuracy_score(y_testing, y_pred)
print("The accuracy of the validation data is {}".format(validate_accuracy))

The cross-validation model score was 0.8063063063063063
0.8183481090786667
{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2090909090909091, 'n_estimators': 10, 'subsample': 1.0}
The accuracy of the validation data is 0.7847533632286996


#### 4.1.2. Predicting

We can now use our fitted model on the 418 rows of test data. This will then be submitted to Kaggle 

In [114]:
xgboost_pred = model.predict(full_test)
pred = xgboost_pred.astype(int)

Finally, we can match the predictions to the passenger Ids and prepare for submission

In [120]:
# extracting passenger IDs
test = pd.read_csv('test.csv')
PassengerId = test.PassengerId

# combining predictions and passenger IDs
xgb = pd.DataFrame({'PassengerId': PassengerId.to_numpy(), 'Survived' : pred})

We can now save this as a csv and submit to kaggle

In [123]:
# saving as a csv
xgb.to_csv('xgb.csv', index = False)

Using this model, we were able to obtain an accuracy of ~77% on the unseen data. We will attempt other models to see how they perform 

### 4.2. Random Forest 

#### 4.2.1. Fitting model

In [17]:
# setting parameters for hyperparameter tuning
params = {
    "min_samples_split": [2, 5, 10, 15, 100],
    "min_samples_leaf": [1, 2, 5, 10],
    "max_features":["auto", "log2", "sqrt"],
    "criterion": ["gini", "entropy"],
    "n_estimators":[50, 100, 300, 500, 800]
    }

# fitting the model
model = GridSearchCV(RandomForestClassifier(), params, cv = 5, n_jobs = -1)
model.fit(x_training, y_training)
print("The cross-validation model score was {}".format(model.score(x_training, y_training)))
print(model.best_score_)
print(model.best_params_)

# validation
y_pred = model.predict(x_testing)
validate_accuracy = accuracy_score(y_testing, y_pred)
print("The accuracy of the validation data is {}".format(validate_accuracy))

The cross-validation model score was 0.9459459459459459
0.8437436875771518
{'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}


#### 4.2.2. Predicting

In [20]:
rf_pred = model.predict(full_test)
pred = rf_pred.astype(int)

In [21]:
# extracting passenger IDs
test = pd.read_csv('test.csv')
PassengerId = test.PassengerId

# combining predictions and passenger IDs
rf = pd.DataFrame({'PassengerId': PassengerId.to_numpy(), 'Survived' : pred})

In [22]:
rf.to_csv('rf.csv', index = False)